In [141]:
import os, sys, json, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
from datetime import date
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file

In [142]:
# What Year?
year = "2024"


In [143]:
#get all leg files


#gather all legislator files from done folder
#committee data doesnt need to be pulled for this

#grab directory for data
dir_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data'

full_path = os.path.join(dir_path,year)
# print(full_path)


#gets most recent file in each folder
os.chdir(full_path)
leg_files = [get_recent_file(f'{f}_legislators*', full_path) for f in next(os.walk('.'))[1]]


## trouble shooting print lines
# print((leg_files))
[print(x.split('\\')[-1]) for x in leg_files]

AL_legislators.xlsx
CT_legislators.xlsx
IL_legislators.xlsx
IN_legislators.xlsx
KS_legislators.xlsx
MO_legislators.xlsx
NC_legislators.xlsx
ND_legislators.xlsx
NM_legislators.xlsx
OH_legislators.xlsx
OK_legislators.xlsx
VA_legislators.xlsx
WV_legislators_2024_11_22.xlsx


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [144]:
#file validation

#quick check to make sure files are found and are excel files
import os
for file in leg_files:
    

    if not os.path.isfile(file):
        raise FileNotFoundError(f"File not found: {file}")

    if not file.endswith('.xlsx'):
        raise ValueError(f"Invalid file format. Expected an .xlsx file, got: {file.split('.')[-1]}")
    


In [145]:

#file compilation

"""compiles legislator files into one file
goes through each sheet and retrieves sheet as dataframe"""

dfs = {}
for i,file in enumerate(leg_files):
    
    ##uncomment the lines below for troubleshooting
    # print('working on file:' + str(file))
    # file = legislator_files[0]
    # xls = pd.ExcelFile(file)
    ##

    sheets_dict = pd.read_excel(file, engine="openpyxl", sheet_name=None)
    sheet_names = list(sheets_dict.keys())
    for s in sheet_names:
        df = pd.read_excel(file, engine="openpyxl", sheet_name=s)
        print(*df.columns, sep = " | ")
        
        filename =  f'{s}'
        dfs[filename] = df


State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader
State Abbreviation | Chamber | full title | First Name | Last Na

In [125]:

## Legislator Data Cleaning
#extract and replace district
for k,v in dfs.items():
    # display_markdown(f'## {k}', raw = True)
    # print(*v.columns, sep=", ")
    print(v.head(2).to_string())
    cols = v.columns.to_list()
    for c in cols:
        if 'District' in c:
            d_col = 'District'
            d_col_alt = 'district'
            break
        elif 'district' in c:
            d_col = 'district'
            break
        else:
            continue
            
    
    v[d_col] = v[d_col].astype(str)
    v[d_col] = v[d_col].str.extractall(r"(\d+)").unstack().fillna('').apply(' '.join, 1)

    if d_col == "District":
        v.rename(columns={d_col:d_col_alt}, inplace=True)
    # print(v.columns)
    dfs[k] = v





  State Abbreviation Chamber                   Title First Name Last Name       Party                 District  Date Assumed Office                                                                                                                           Committee List                               Name  tenure leader
0                 AL   House  Alabama Representative    Cynthia    Almond  Republican  State House District 63                 2021                                    Ethics and Campaign Finance (Vice Chair), Rules, Judiciary, Ways and Means Education, Reapportionment  AL Rep. Cynthia Almond (R-AL-063)       4    NaN
1                 AL   House  Alabama Representative       Alan     Baker  Republican  State House District 66                 2006  Local Legislation (Chair), Education Policy (Vice Chair), Ways and Means Education, Baldwin County Legislation, Technology and Research      AL Rep. Alan Baker (R-AL-066)      19    NaN
  State Abbreviation Chamber            Tit

In [126]:

#create pk with no seats
dfs_w_pk = {}
emptys = {}

for k,v in dfs.items():
    # display_markdown(f'## {k}', raw = True)
    # print(v.columns)
    # print(type(v))
    if v.empty:
        print('1')
        print('intitial is empty?')
        break
    # v = v.drop(['Committee List', 'Date Assumed Office'], axis=1)
    
    #call funciton to get pk
    cleaned_df, dupes = create_pk(v,'district', 'Chamber')
    # print(type(cleaned_df))
    # print(type(dupes))
    dfs_to_review = [cleaned_df,dupes]
    
    for d in dfs_to_review:
        if cleaned_df.empty:
            dfs_to_review = [df for df in dfs_to_review if not df.equals(cleaned_df)]
        if dupes.empty:
            dfs_to_review = [df for df in dfs_to_review if not df.equals(dupes)]

    
    df = pd.concat(dfs_to_review).reset_index(drop = True)
    df.to_dict(orient='index')
    print(df.head(2))

    dfs_w_pk[k] = df
    
    
    


duplicate df is empty (from function)
  primary_key district_code State Abbreviation Chamber  \
0      100063           063                 AL   House   
1      100066           066                 AL   House   

                    Title First Name Last Name       Party district  \
0  Alabama Representative    Cynthia    Almond  Republican       63   
1  Alabama Representative       Alan     Baker  Republican       66   

   Date Assumed Office                                     Committee List  \
0                 2021  Ethics and Campaign Finance (Vice Chair), Rule...   
1                 2006  Local Legislation (Chair), Education Policy (V...   

                                Name  tenure leader state_code chamber_code  
0  AL Rep. Cynthia Almond (R-AL-063)       4    NaN         10            0  
1      AL Rep. Alan Baker (R-AL-066)      19    NaN         10            0  
duplicate df is empty (from function)
  primary_key district_code State Abbreviation Chamber            Tit

In [127]:
#save all dfs for use in influence

if year != "2024":
    for k,v in dfs_w_pk.items():
        v.to_csv(fr'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\bridges\dfs_w_pks\{k}.csv', index=False)

In [128]:
#creates standard leg files without extra info
#trims files to not include committee data
compiling = {}
for k,v in dfs_w_pk.items():
    print(k)
    print(v.head(2))
    # print(v.columns)
    # print(len(v))
    try:
        df = v.drop(['Committee List'], axis=1)
        
        compiling[k] = df
        #print(k," is in")
    except:
        df = v
        compiling[k] = df
        



AL_house
  primary_key district_code State Abbreviation Chamber  \
0      100063           063                 AL   House   
1      100066           066                 AL   House   

                    Title First Name Last Name       Party district  \
0  Alabama Representative    Cynthia    Almond  Republican       63   
1  Alabama Representative       Alan     Baker  Republican       66   

   Date Assumed Office                                     Committee List  \
0                 2021  Ethics and Campaign Finance (Vice Chair), Rule...   
1                 2006  Local Legislation (Chair), Education Policy (V...   

                                Name  tenure leader state_code chamber_code  
0  AL Rep. Cynthia Almond (R-AL-063)       4    NaN         10            0  
1      AL Rep. Alan Baker (R-AL-066)      19    NaN         10            0  
AL_senate
  primary_key district_code State Abbreviation Chamber            Title  \
0      101022           022                 AL  Sen

In [129]:
for k,v in compiling.items():
    print('############')
    print(k)
    print(v.head(2))

############
AL_house
  primary_key district_code State Abbreviation Chamber  \
0      100063           063                 AL   House   
1      100066           066                 AL   House   

                    Title First Name Last Name       Party district  \
0  Alabama Representative    Cynthia    Almond  Republican       63   
1  Alabama Representative       Alan     Baker  Republican       66   

   Date Assumed Office                               Name  tenure leader  \
0                 2021  AL Rep. Cynthia Almond (R-AL-063)       4    NaN   
1                 2006      AL Rep. Alan Baker (R-AL-066)      19    NaN   

  state_code chamber_code  
0         10            0  
1         10            0  
############
AL_senate
  primary_key district_code State Abbreviation Chamber            Title  \
0      101022           022                 AL  Senate  Alabama Senator   
1      101021           021                 AL  Senate  Alabama Senator   

  First Name  Last Name    

In [130]:

#pull togther all newly trimmed df's

all_legs_files = pd.concat(list(compiling.values())).reset_index(drop = True)
# all_legs_files = all_legs_files.drop_duplicates().reset_index(drop = True)
all_legs_files


,primary_key,district_code,State Abbreviation,Chamber,Title,First Name,Last Name,Party,district,Date Assumed Office,Name,tenure,leader,state_code,chamber_code
0,100063,063,AL,House,Alabama Representative,Cynthia,Almond,Republican,63,2021,AL Rep. Cynthia Almond (R-AL-063),4,NaN,10,0
1,100066,066,AL,House,Alabama Representative,Alan,Baker,Republican,66,2006,AL Rep. Alan Baker (R-AL-066),19,NaN,10,0
2,100049,049,AL,House,Alabama Representative,Russell,Bedsole,Republican,49,2020,AL Rep. Russell Bedsole (R-AL-049),5,NaN,10,0
3,100080,080,AL,House,Alabama Representative,Chris,Blackshear,Republican,80,2016,AL Rep. Chris Blackshear (R-AL-080),9,NaN,10,0
4,100061,061,AL,House,Alabama Representative,Ronald,Bolton,Republican,61,2022,"AL Rep. Ronald ""Ron"" Bolton (R-AL-061)",3,NaN,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,571015,015,WV,Senate,West Virginia Senator,Darren,Thorne,Republican,15,2025,WV Sen. Darren Thorne (R-WV-015),0,NaN,57,1
1981,571001,001,WV,Senate,West Virginia Senator,Ryan,Weld,Republican,1,2016,WV Sen. Ryan Weld (R-WV-001),9,NaN,57,1
1982,571015,015,WV,Senate,West Virginia Senator,Thomas,Willis,Republican,15,2024,"WV Sen. Thomas ""Tom"" Willis (R-WV-015)",1,NaN,57,1
1983,571005,005,WV,Senate,West Virginia Senator,Michael,Woelfel,Democrat,5,2024,"WV Sen. Michael ""Mike"" Woelfel (D-WV-005)",1,NaN,57,1


In [ ]:


# os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files')
legs_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files'


save_path = os.path.join(legs_path, year)
file_name = f'{year}_all_legs_files_{str(date.today()).replace('-','_')}.xlsx'
full_path = os.path.join(save_path, file_name)



try:
    all_legs_files.to_excel(full_path, index=False)
except:
    print(full_path)
    



# print(*all_legs_files.columns, sep = " | ")
all_legs_files


# all legs records

In [132]:


#pull together all years all_legs data


path_24 = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2024"
path_25 = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2025"

file_24 = get_recent_file(r'*.xlsx',path_24)
file_25 = get_recent_file(r'*.xlsx',path_25)

print(file_24)
print(file_25)

df_24 = pd.read_excel(file_24)

df_25 = pd.read_excel(file_25)


C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2024\2024_all_legs_files_2025_02_04.xlsx
C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2025\2025_all_legs_files_2025_02_04.xlsx


In [134]:
print(*df_24.columns, sep = " | ")
print(*df_25.columns, sep = " | ")


primary_key | district_code | State Abbreviation | Chamber | full title | First Name | Last Name | Party | district | tenure | leader | state_code | chamber_code | Title | Date Assumed Office
primary_key | district_code | State Abbreviation | Chamber | Title | First Name | Last Name | Party | district | Date Assumed Office | Name | tenure | leader | state_code | chamber_code


In [65]:

# print(len(df_24.columns))

#take out name from 2025 data and move district to the back
column_to_move = df_24.pop('district')
df_24.insert(14, 'district', column_to_move)
# print(df_25.columns)
column_to_move = df_25.pop('district')
df_25.insert(14, 'district', column_to_move)


df_25 = df_25.drop('Name', axis='columns')

print(*df_24.columns, sep = ' ')
print(*df_25.columns, sep = ' ')



primary_key district_code State Abbreviation Chamber Title First Name Last Name Party Date Assumed Office Committee List tenure leader state_code chamber_code district
primary_key district_code State Abbreviation Chamber Title First Name Last Name Party Date Assumed Office tenure leader state_code chamber_code district


primary_key district_code State Abbreviation Chamber Title First Name Last Name Party Date Assumed Office Committee List tenure leader state_code chamber_code district
primary_key district_code State Abbreviation Chamber Title First Name Last Name Party district Date Assumed Office tenure leader state_code chamber_code


In [ ]:

df_24['recorded_year'] = "2024"
df_25['recorded_year'] = "2025"

all_leg_records = pd.concat([df_24, df_25])
all_leg_records = all_leg_records.reset_index(drop = False)



all_leg_records.to_excel(fr'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\all_leg_records_v2.xlsx')
all_leg_records


In [ ]:

## compile working files
#creates standard leg files without extra info
#trims files to not include committee data
compiling = []
for k,v in dfs_w_pk.items():
    
    # print(k)
    # print(v.columns)

    print(len(v))
    try:
        df = v.drop(['Committee List'], axis=1)
        # print(*df.columns, sep = " | ")
        compiling.append(df)
        #print(k," is in")
    except:
        compiling.append(df)




In [ ]:
    
#pull togther all newly trimmed df's
all_legs_files = pd.concat(compiling).reset_index(drop = True)
all_legs_files = all_legs_files.drop_duplicates().reset_index(drop = True)


os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files')
legs_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files'
year = "2024"

save_path = os.path.join(legs_path, year)
file_name = f'{year}_all_legs_files_{str(date.today()).replace('-','_')}.xlsx'
full_path = os.path.join(save_path, file_name)

try:
    all_legs_files.to_excel(full_path, index=False)
except:
    print(full_path)
    




# print(*all_legs_files.columns, sep = " | ")
all_legs_files



In [ ]:

#pull together all years all_legs data

file_2024 = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2024\2024_all_legs_files_2025_01_27.xlsx"
file_2025 = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\2025\all_legs_files_2025_01_21.xlsx"

df_24 = pd.read_excel(file_2024)
df_25 = pd.read_excel(file_2025)

print(len(df_24.columns))


column_to_move = df_24.pop('district')
df_24.insert(15, 'district', column_to_move)
# print(df_25.columns)
df_25 = df_25.drop('Name', axis='columns')



print(*df_24.columns, sep = ' ')
print(*df_25.columns, sep = ' ')

df_24['recorded_year'] = "2024"
df_25['recorded_year'] = "2025"

all_leg_records = pd.concat([df_24, df_25])
all_leg_records = all_leg_records.reset_index(drop = False)

# all_leg_records.to_excel(fr'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\all_leg_records.xlsx')

